In [1]:
import io
import os

from google.cloud import vision

%load_ext dotenv
%dotenv

In [2]:
def combine_words(ocr_results, proximity_threshold=5):
    combined_results = []
    current_phrase = ""
    current_box = None
    last_word_end = None

    for i, result in enumerate(ocr_results):
        if i == 0: continue
        # Get the start (x_min) and end (x_max) of the bounding box for the current word
        vertices = ([(vertex.x, vertex.y)
                    for vertex in result.bounding_poly.vertices])
        x_min = vertices[0][0]
        x_max = vertices[1][0]

        if last_word_end is None or \
            (x_min - last_word_end) <= proximity_threshold:
            # Append to the current phrase
            current_phrase += result.description
            # Extend the current bounding box
            if current_box is None:
                current_box = result.bounding_poly
            else:
                current_box.vertices[1].x = vertices[1][0]
                current_box.vertices[2].x = vertices[2][0]
        else:
            # Start a new phrase and bounding box
            combined_results.append((current_phrase, current_box))
            current_phrase = result.description
            current_box = result.bounding_poly
        
        # Update the end position of the last word
        last_word_end = x_max

    # Don't forget to add the last phrase and box
    if current_phrase:
        combined_results.append((current_phrase, current_box))
    
    return combined_results

In [6]:
def detect_text_with_combine(path):
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    try:
        image = vision.Image(content=content)

        response = client.text_detection(image=image)
        texts = response.text_annotations

        texts = combine_words(texts, 5)
        # print([item[0] for item in test])
        # print([bbox[1] for bbox in test])
        
        return texts
    except Exception as e:
        raise Exception(f"Error: {e}")
    
def detect_text(path):
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    try:
        image = vision.Image(content=content)

        response = client.text_detection(image=image)
        texts = response.text_annotations

        # texts = combine_words(texts, 5)
        # print([item[0] for item in test])
        # print([bbox[1] for bbox in test])
        
        return texts
    except Exception as e:
        raise Exception(f"Error: {e}")

In [4]:
result = detect_text("./test11.png")

[('[섭취량',
  vertices {
    x: 7
    y: 9
  }
  vertices {
    x: 74
    y: 9
  }
  vertices {
    x: 74
    y: 31
  }
  vertices {
    x: 7
    y: 31
  }),
 ('및',
  vertices {
    x: 87
    y: 9
  }
  vertices {
    x: 104
    y: 9
  }
  vertices {
    x: 104
    y: 31
  }
  vertices {
    x: 87
    y: 31
  }),
 ('섭취방법]',
  vertices {
    x: 119
    y: 9
  }
  vertices {
    x: 205
    y: 9
  }
  vertices {
    x: 205
    y: 31
  }
  vertices {
    x: 119
    y: 31
  }),
 ('1일',
  vertices {
    x: 220
    y: 9
  }
  vertices {
    x: 246
    y: 9
  }
  vertices {
    x: 246
    y: 31
  }
  vertices {
    x: 220
    y: 31
  }),
 ('1회,',
  vertices {
    x: 262
    y: 9
  }
  vertices {
    x: 296
    y: 9
  }
  vertices {
    x: 296
    y: 31
  }
  vertices {
    x: 262
    y: 31
  }),
 ('1회',
  vertices {
    x: 312
    y: 9
  }
  vertices {
    x: 338
    y: 9
  }
  vertices {
    x: 338
    y: 31
  }
  vertices {
    x: 312
    y: 31
  }),
 ('1포를',
  vertices {
    x: 356
    y: 9
 

In [7]:
from openai import OpenAI

client = OpenAI()

result = detect_text("./test11.png")
completion = client.chat.completions.create(
  model="gpt-4-0125-preview",
  messages=[
    {"role": "system", "content": "너는 OCR Recognizer결과를 받아서 한국어 의미 단위로 단어를 합쳐주는 Serializer야. 데이터를 받으면 해당 데이터를 의미 단위로 Serialize해주고, 맞춤법이 틀린 부분이 있다면 해당하는 bbox 좌표와 함께 어디가 틀렸고, 어떻게 고쳐야하는지 알려줘"},
    {"role": "user", "content": f"데이터는 이거야. {result}"}
  ]
)

print("------- 자체 Serialize 없이 Google Vision 결과 그대로 입력 -------") 
print(completion.choices[0].message)

result = detect_text_with_combine("./test11.png")
completion = client.chat.completions.create(
  model="gpt-4-0125-preview",
  messages=[
    {"role": "system", "content": "너는 OCR Recognizer결과를 받아서 한국어 의미 단위로 단어를 합쳐주는 Serializer야. 데이터를 받으면 해당 데이터를 의미 단위로 Serialize해주고, 맞춤법이 틀린 부분이 있다면 해당하는 bbox 좌표와 함께 어디가 틀렸고, 어떻게 고쳐야하는지 알려줘"},
    {"role": "user", "content": f"데이터는 이거야. {result}"}
  ]
)
print("------- 자체 Serialize 통과 후 입력 -------") 
print(completion.choices[0].message)

------- 자체 Serialize 없이 Google Vision 결과 그대로 입력 -------
ChatCompletionMessage(content='직렬화된 텍스트와 맞춤법 검토 결과는 다음과 같습니다.\n\n직렬화된 텍스트:\n```\n"[키워드 목록] 1월 1일, 1일 1명은 키워드로 검색해보세요. [키워드 검색 방법]. 먼저, 고객\\n서비스를 이용하여 문의사항이 있으면 담당자에게 연락하여 해결하는 것이 좋습니다. 직접\\n문의하시려면 고객센터에 전화하시거나 홈페이지를 통해 문의할 수 있습니다.\\n정기적으로 키워드를 검색하고 상위노출을 위해 노력하세요\\n포털사이트에서 키워드로 검색하여 찾아볼 수 있습니다.\\n키워드를 잘 선택하면 검색 효율이 높아집니다.\\n\\n\\n철저히 준비된 자료로 키워드를 구체적으로 알아보세요. 상위노출을 위한 최적의 방법을 찾아보세요.\\n\\n\\n먼저, 고객님의 요구사항에 맞는 키워드를 선택하세요.\\n"\n```\n\n맞춤법 검토 결과:\n1. `"키워드로 검색해보세요"` - 맞춤법에 문제 없음.\n2. `"먼저, 고객 서비스를 이용하여 문의사항이 있으면 담당자에게 연락하여 해결하는 것이 좋습니다."` - `"고객 서비스"`에 대한 띄어쓰기는 올바르며, 전체 문장에 대한 맞춤법에 문제가 없음. \n3. `"정기적으로 키워드를 검색하고 상위노출을 위해 노력하세요"` - 맞춤법에 문제 없음.\n4. `"포털사이트에서 키워드로 검색하여 찾아볼 수 있습니다."` - 맞춤법에 문제 없음.\n5. `"키워드를 잘 선택하면 검색 효율이 높아집니다."` - 맞춤법에 문제 없음.\n6. `"철저히 준비된 자료로 키워드를 구체적으로 알아보세요."` - 맞춤법에 문제 없음.\n7. `"상위노출을 위한 최적의 방법을 찾아보세요."` - 맞춤법에 문제 없음.\n8. `"먼저, 고객님의 요구사항에 맞는 키워드를 선택하세요."` - 맞춤법에 문제 없음.\n\n이 텍스트 집합에는 맞춤법이나 띄어쓰기에서 분명한 오류가 없어 보입니다. 데이터

In [8]:
result = detect_text_with_combine("./test11.png")
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "너는 OCR Recognizer 결과를 받아서 한국어 의미 단위로 단어를 합쳐주는 Serializer야. 데이터를 받으면 해당 데이터를 의미 단위로 Serialize해주고, 맞춤법이 틀린 부분이 있다면 해당하는 bbox 좌표와 함께 어디가 틀렸고, 어떻게 고쳐야하는지 알려줘"},
    {"role": "user", "content": f"데이터는 이거야. {result}"}
  ]
)
print("------- 자체 Serialize 통과 후 gpt-3.5-turbo에 입력 -------") 
print(completion.choices[0].message)

------- 자체 Serialize 통과 후 gpt-3.5-turbo에 입력 -------
ChatCompletionMessage(content="잘못된 부분:\n1. '섭취량' => '섭취량'\n2. '섭취방법]' => '섭취방법]'\n3. '1회,' => '1회,'\n4. '특이체질,' => '특이체질,'\n5. '알레르기체질' => '알레르기체질'\n6. '및' => '및'\n7. '시' => '시'\n8. '등은' => '등은'\n9. '함부로' => '함부로'\n10. '않도록질환이' => '않도록 질환이'\n11. '일일섭취양' => '일일 섭취량'\n\n이렇게 수정이 필요합니다.", role='assistant', function_call=None, tool_calls=None)
